# TOPIC MODELING

### Dependencies

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import string, json, glob, re
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

### Loading data

In [3]:
data = pd.read_excel('metoo_tweets_dec2017.xlsx')
data.head()

,column_a,text,favorited,favoritecount,replytosn,created,truncated,replytosid,id,replytouid,statussource,screenname,retweetcount,isretweet,retweeted,longitude,latitude,location
0,1,American Harem.. #MeToo https://t.co/HjExLJdGuF,0.0,0,NaN,2017-11-29T23:59:00,0.0,NaN,9.360000e+17,NaN,"<a href=""http://instagram.com"" rel=""nofollow"">...",ahmediaTV,0.0,0.0,0.0,NaN,NaN,NaN
1,2,@johnconyersjr @alfranken why have you guys ...,0.0,0,johnconyersjr,2017-11-29T23:59:00,0.0,NaN,9.360000e+17,266149840,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",JesusPrepper74,0.0,0.0,0.0,NaN,NaN,NaN
2,3,Watched Megan Kelly ask Joe Keery this A.M. if...,0.0,0,NaN,2017-11-29T23:59:00,1.0,NaN,9.360000e+17,NaN,"<a href=""http://twitter.com/download/android"" ...",DemerisePotvin,0.0,0.0,0.0,NaN,NaN,NaN
3,4,Women have been talking about this crap the en...,0.0,0,NaN,2017-11-29T23:59:00,0.0,NaN,9.360000e+17,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",TheDawnStott,0.0,0.0,0.0,NaN,NaN,NaN
4,5,.@BetteMidler please speak to this sexual assa...,0.0,15,NaN,2017-11-29T23:59:00,0.0,NaN,9.360000e+17,NaN,"<a href=""http://twitter.com/#!/download/ipad"" ...",scottygirl2014,11.0,0.0,0.0,NaN,NaN,NaN


### Cleaning Data

In [4]:
# Removing the false and nanvalues
filtered_data = data[data["text"].apply(lambda x: type(x) == str)]

def clean_text(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'RT[\s]+', '', text)
    text = text.lower()
    text = re.sub("#[A-Za-z0-9_]+","", text)
    text = re.sub(r'@[A-Za-z0-9]+', '', text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    
    text =  ' '.join([word for word in text.split() if word.lower() not in stop_words])
    return text

filtered_data['clean_text'] = filtered_data['text'].apply(clean_text)



C:\Users\Patil\AppData\Local\Temp\ipykernel_36296\2740783362.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['clean_text'] = filtered_data['text'].apply(clean_text)


In [8]:
df2 = filtered_data.get('clean_text')

vectorizer = TfidfVectorizer(
    lowercase = True,
    max_features = 100, # top 100 topics
    max_df = 0.8,  # Ignore the topics with more than 80 percent 
    min_df = 10, #ignore the terms which has frequency lower than 10 
    ngram_range = (1,3),
    stop_words = "english"
)

vectors = vectorizer.fit_transform(df2)
feature_names = vectorizer.get_feature_names_out()

dense = vectors.todense()
denselist = dense.tolist()


df = pd.DataFrame(denselist, columns=feature_names)


# all_keywords = []
# for description in denselist:
#     x=0
#     keywords = []
#     for word in description:
#         if word > 0:
#             keywords.append(feature_names[x])
#         x=x+1
#     all_keywords.append(keywords)


# print(all_keywords)






In [9]:
df

,2017,abuse,ago,al,allegations,allegations president,allegations president united,allen,america,amp,...,x000d,x000dx000d,year,years,years ago,young,½ï,½ï ½ï,½ï ½ï ½ï,½ûªs
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
393112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
393113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
393114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
